In [58]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import differential_evolution
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [59]:
df = pd.read_excel('各参数SEA.xlsx')

In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df[['α(°)',  'β(°)',  'l1/l2',  't_mm',]]),
    np.array(df[['SEA']]),
    test_size=0.2,
    shuffle=True,
    random_state=42
)

In [61]:
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(f'xmean{X_scaler.mean_},\nxvar{X_scaler.var_}')

Y_scaler = StandardScaler()
y_train_scaled = Y_scaler.fit_transform(y_train)
y_test_scaled = Y_scaler.transform(y_test)
print(f'ymean{Y_scaler.mean_}, \nyvar{Y_scaler.var_}')

xmean[45.30133929 54.86607143  2.26004464  1.15223214],
xvar[9.78444625e+01 4.71918846e+01 3.12399105e-01 1.23164461e-02]
ymean[670.66526786], 
yvar[22505.59169011]


In [63]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# 创建样本权重，给高SEA值更大权重
sample_weights = np.where(
    y_train > np.percentile(y_train, 90),  # 高于90分位数的样本
    2.0,  # 权重加倍
    1.0   # 正常权重
)

history = model.fit(
    X_train_scaled, y_train_scaled,
    sample_weight=sample_weights.flatten(),
    epochs=200,
    verbose=0
)

In [64]:
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test_scaled, y_pred))
r2_test = r2_score(y_test_scaled, y_pred)
r2_train = r2_score(y_train_scaled, model.predict(X_train_scaled))
print(rmse)
print(r2_test)
print(r2_train)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
0.14499772074052672
0.9806624650898629
0.9831853155346236


In [65]:
data = df.loc[df.SEA == 1023.07]
data1 = X_scaler.transform(data.iloc[:,0:4])
data2 = Y_scaler.inverse_transform(model.predict(data1))
print(data1)
print(data2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
[[-1.04141916  0.74733644 -1.35982873  1.33148804]]
[[952.71277]]


D:\dev\Anaconda3\envs\moo2\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [33]:
def objective(x):
    # 将输入转换为模型需要的形状并标准化
    x_reshaped = x.reshape(1, -1)
    x_scaled = X_scaler.transform(x_reshaped)
    # 预测并反标准化
    y_pred_scaled = model.predict(x_scaled, verbose=0)
    y_pred = Y_scaler.inverse_transform(y_pred_scaled)
    return -y_pred[0][0]  # 返回负值用于最小化任务

# 2. 设置搜索边界 (根据你数据集的范围设定)
bounds = [
    (df['α(°)'].min(), df['α(°)'].max()),
    (df['β(°)'].min(), df['β(°)'].max()),
    (df['l1/l2'].min(), df['l1/l2'].max()),
    (df['t_mm'].min(), df['t_mm'].max())
]

# 3. 执行全局寻优
result = differential_evolution(objective, bounds, strategy='best1bin', maxiter=100, popsize=15)

print("--- 最优化结果 ---")
print(f"最大 SEA 值预测: {-result.fun:.2f}")
print(f"最佳参数组合: ")
print(f" α: {result.x[0]:.2f}, β: {result.x[1]:.2f}, l1/l2: {result.x[2]:.2f}, t_mm: {result.x[3]:.2f}")

--- 最优化结果 ---
最大 SEA 值预测: 953.27
最佳参数组合: 
 α: 31.30, β: 64.80, l1/l2: 1.54, t_mm: 1.30
